In [ ]:
def predict_winner(state: np.ndarray, agent_outputs: List[str], context: Dict) -> str:
    """
    Predict winner based on coordinator state.
    
    This is a simple heuristic - in practice, you would train a classifier.
    For this experiment, we use a rule-based approach.
    """
    try:
        # Extract features from agent outputs
        len_a = len(agent_outputs[0])
        len_b = len(agent_outputs[1])
        
        # Use state to weight features
        # Sum of positive state values as "confidence"
        confidence = np.sum(state[state > 0])
        
        # Simple heuristic: longer, more detailed response is often better
        # But weight by coordinator's state confidence
        score_a = len_a * (1 + confidence * 0.1)
        score_b = len_b * (1 + confidence * 0.1)
        
        # If very close, predict tie
        if abs(score_a - score_b) < 50:
            prediction = "tie"
        elif score_a > score_b:
            prediction = "model_a"
        else:
            prediction = "model_b"
        
        logger.debug(f"  Prediction: {prediction} (score_a={score_a:.1f}, score_b={score_b:.1f}, confidence={confidence:.4f})")
        return prediction
        
    except Exception as e:
        logger.error(f"{RED}Error in predict_winner: {e}{END}")
        return "tie"  # Default fallback

def run_experiment(examples: List[Dict], coordinator, tracker: TokenTracker, name: str) -> Tuple[List[str], List[int]]:
    """
    Run experiment with given coordinator.
    
    Args:
        examples: Dataset examples
        coordinator: Coordinator instance (FullRank or LowRank)
        tracker: Token tracker instance  
        name: Experiment name for logging
        
    Returns:
        Tuple of (predictions, token_counts_per_episode)
    """
    try:
        print(f"\n{BLUE}{'='*60}{END}")
        print(f"{BLUE}Running experiment: {name}{END}")
        print(f"{BLUE}{'='*60}{END}")
        
        predictions = []
        token_counts = []
        
        for idx, example in enumerate(examples):
            try:
                print(f"\n{CYAN}Processing example {idx+1}/{len(examples)}{END}")
                logger.debug(f"Input: {truncate_str(example['input'])}")
                
                # Reset coordinator state for new episode
                coordinator.reset()
                
                # Track tokens for this episode
                episode_start_tokens = tracker.total_tokens
                
                # Extract agent outputs from dataset
                context = example['context']
                agent_outputs = [
                    context['response_a'],
                    context['response_b']
                ]
                
                logger.debug(f"Agent A ({context['model_a']}): {truncate_str(agent_outputs[0], 80)}")
                logger.debug(f"Agent B ({context['model_b']}): {truncate_str(agent_outputs[1], 80)}")
                
                # Coordinator processes both agent responses
                state, coordinator_message = coordinator.step(agent_outputs)
                
                # Track tokens (agent outputs + coordinator message)
                step_tokens = tracker.log_coordinator_step(agent_outputs, coordinator_message)
                
                # Make prediction based on coordinator's final state
                # Simple heuristic: compare state activation with agent output lengths
                prediction = predict_winner(state, agent_outputs, context)
                predictions.append(prediction)
                
                # Record tokens for this episode
                episode_tokens = tracker.total_tokens - episode_start_tokens
                token_counts.append(episode_tokens)
                
                print(f"{GREEN}Episode {idx+1} complete: prediction={prediction}, tokens={episode_tokens}{END}")
                
            except Exception as e:
                logger.error(f"{RED}Error processing example {idx+1}: {e}{END}")
                # Add default prediction and token count to continue
                predictions.append("tie")
                token_counts.append(0)
        
        print(f"\n{GREEN}Experiment {name} complete: {len(predictions)} predictions made{END}")
        return predictions, token_counts
        
    except Exception as e:
        logger.error(f"{RED}Error in run_experiment: {e}{END}")
        raise

def evaluate_performance(predictions: List[str], ground_truth: List[str]) -> Dict[str, float]:
    """
    Evaluate task performance metrics.
    
    Args:
        predictions: Predicted winners
        ground_truth: Actual winners from dataset
        
    Returns:
        Dictionary of performance metrics
    """
    try:
        print(f"\n{BLUE}Evaluating performance...{END}")
        
        # Compute accuracy
        accuracy = accuracy_score(ground_truth, predictions)
        print(f"  Accuracy: {accuracy:.4f}")
        
        # Compute F1 scores
        f1_macro = f1_score(ground_truth, predictions, average='macro', zero_division=0)
        f1_weighted = f1_score(ground_truth, predictions, average='weighted', zero_division=0)
        print(f"  F1 (macro): {f1_macro:.4f}")
        print(f"  F1 (weighted): {f1_weighted:.4f}")
        
        metrics = {
            "accuracy": float(accuracy),
            "f1_macro": float(f1_macro), 
            "f1_weighted": float(f1_weighted)
        }
        
        print(f"{GREEN}Performance evaluation complete{END}")
        return metrics
        
    except Exception as e:
        logger.error(f"{RED}Error evaluating performance: {e}{END}")
        return {
            "accuracy": 0.0,
            "f1_macro": 0.0,
            "f1_weighted": 0.0
        }

def compute_statistical_significance(baseline_tokens: List[int], method_tokens: List[int]) -> Dict[str, Any]:
    """
    Test if improvement is statistically significant.
    
    Args:
        baseline_tokens: Token counts per episode for baseline
        method_tokens: Token counts per episode for method
        
    Returns:
        Dictionary of statistical test results
    """
    try:
        print(f"\n{BLUE}Computing statistical significance...{END}")
        
        # Paired t-test (assumes normality)
        t_stat, t_pval = ttest_rel(baseline_tokens, method_tokens)
        t_significant = t_pval < 0.05
        
        print(f"  Paired t-test: t={t_stat:.4f}, p={t_pval:.4f}, significant={t_significant}")
        
        # Effect size (Cohen's d)
        mean_diff = np.mean(baseline_tokens) - np.mean(method_tokens)
        pooled_std = np.sqrt((np.var(baseline_tokens) + np.var(method_tokens)) / 2)
        cohens_d = mean_diff / pooled_std if pooled_std > 0 else 0
        
        print(f"  Effect size (Cohen's d): {cohens_d:.4f}")
        
        results = {
            "t_statistic": float(t_stat),
            "t_pvalue": float(t_pval),
            "t_significant": bool(t_significant),
            "cohens_d": float(cohens_d),
            "alpha": 0.05
        }
        
        print(f"{GREEN}Statistical tests complete{END}")
        return results
        
    except Exception as e:
        logger.error(f"{RED}Error computing statistical significance: {e}{END}")
        return {
            "t_statistic": 0.0,
            "t_pvalue": 1.0,
            "t_significant": False,
            "cohens_d": 0.0,
            "alpha": 0.05
        }

print("🔧 Experimental framework ready!")

## Experimental Framework

Now we'll set up the experimental framework to run and compare both coordinators.

In [ ]:
class LowRankRecurrentCoordinator:
    """
    Method: Low-rank recurrent coordinator with RIM-inspired sparse recurrence.

    Key innovations:
    1. Low-rank factorization: W = U @ V^T (reduces parameters)
    2. Sparse recurrence: Only k modules active per step (reduces computation)
    3. Compressed messaging: Messages encode only rank dimensions (reduces communication)
    """

    def __init__(self, hidden_dim: int = 256, rank: int = 32, num_modules: int = 4):
        """
        Initialize low-rank coordinator.

        Args:
            hidden_dim: Dimension of hidden state
            rank: Low-rank factorization rank (k << hidden_dim)
            num_modules: Number of RIM modules for sparse recurrence
        """
        self.hidden_dim = hidden_dim
        self.rank = rank
        self.num_modules = num_modules

        # Low-rank factorization: W = U @ V^T
        self.U = np.random.randn(hidden_dim, rank) * 0.01
        self.V = np.random.randn(hidden_dim, rank) * 0.01

        # RIM sparse attention: only k modules active per step
        self.active_k = max(1, num_modules // 2)

        # Module-specific transformations (lightweight)
        self.module_weights = [
            np.random.randn(hidden_dim, rank) * 0.01
            for _ in range(num_modules)
        ]

        self.state = np.zeros(hidden_dim)

        compression_ratio = rank / hidden_dim
        param_reduction = (2 * hidden_dim * rank) / (hidden_dim * hidden_dim)

        logger.info(f"{GREEN}LowRankRecurrentCoordinator initialized:{END}")
        logger.info(f"  hidden_dim={hidden_dim}, rank={rank}, num_modules={num_modules}")
        logger.info(f"  Compression ratio: {compression_ratio:.2%}")
        logger.info(f"  Parameter reduction: {param_reduction:.2%} of full-rank")
        logger.info(f"  Active modules per step: {self.active_k}/{num_modules}")

    def reset(self):
        """Reset coordinator state."""
        self.state = np.zeros(self.hidden_dim)
        logger.debug("Coordinator state reset")

    def step(self, agent_outputs: List[str]) -> Tuple[np.ndarray, str]:
        """
        Low-rank recurrent step with sparse module updates.

        Args:
            agent_outputs: List of agent response strings

        Returns:
            Tuple of (updated state, coordinator message)
        """
        try:
            logger.debug(f"LowRankCoordinator.step with {len(agent_outputs)} agent outputs")

            # Encode agent outputs
            features = self._encode_outputs(agent_outputs)
            logger.debug(f"  Encoded features shape: {features.shape}")

            # Select active modules (RIM sparse recurrence)
            active_modules = self._select_active_modules(self.state, features)
            logger.debug(f"  Active modules: {active_modules}")

            # Low-rank recurrent update: s_{t+1} = U @ (V^T @ s_t) + module_updates
            # This is equivalent to W @ s_t where W = U @ V^T, but much cheaper
            state_proj = self.V.T @ self.state  # Project to rank-dimensional space
            logger.debug(f"  State projection shape: {state_proj.shape}")

            new_state = self.U @ state_proj  # Reconstruct in original space

            # Apply sparse module updates (only active modules)
            for module_idx in active_modules:
                module_update = self.module_weights[module_idx] @ state_proj
                new_state += module_update
                logger.debug(f"    Module {module_idx} update norm: {np.linalg.norm(module_update):.4f}")

            # Add features
            new_state += features

            self.state = new_state
            logger.debug(f"  State norm after update: {np.linalg.norm(self.state):.4f}")

            # Generate compressed coordinator message
            # In low-rank, message only encodes rank dimensions (lower communication cost)
            coordinator_message = self._generate_compressed_message(state_proj)
            logger.debug(f"  Coordinator message length: {len(coordinator_message)}")

            return self.state.copy(), coordinator_message

        except Exception as e:
            logger.error(f"{RED}Error in LowRankCoordinator.step: {e}{END}")
            raise

    def _encode_outputs(self, outputs: List[str]) -> np.ndarray:
        """Encode agent outputs into feature vector."""
        try:
            # Simple encoding: token counts + basic text statistics
            features = []
            for output in outputs:
                features.extend([
                    len(output.split()),  # Word count
                    len(output),  # Character count
                    output.count('.'),  # Sentence markers
                    output.count('?'),  # Question markers
                ])

            # Pad or truncate to hidden_dim
            features_array = np.array(features[:self.hidden_dim])
            if len(features_array) < self.hidden_dim:
                padded = np.zeros(self.hidden_dim)
                padded[:len(features_array)] = features_array
                features_array = padded

            # Normalize
            features_array = features_array / (np.linalg.norm(features_array) + 1e-8)

            logger.debug(f"  Encoded {len(outputs)} outputs into {len(features_array)} features")
            return features_array

        except Exception as e:
            logger.error(f"{RED}Error encoding outputs: {e}{END}")
            return np.zeros(self.hidden_dim)

    def _select_active_modules(self, state: np.ndarray, features: np.ndarray) -> List[int]:
        """
        Select top-k modules based on attention scores (RIM mechanism).

        Args:
            state: Current hidden state
            features: Encoded input features

        Returns:
            List of active module indices
        """
        try:
            # Compute attention scores for each module
            scores = []
            state_proj = self.V.T @ state  # Project to rank space

            for module_idx, module_w in enumerate(self.module_weights):
                # Score = similarity between module projection and features
                module_proj = module_w.T @ features
                score = np.dot(module_proj[:self.rank], state_proj)
                scores.append((score, module_idx))

            # Return top-k active modules
            scores.sort(reverse=True)
            active = [idx for _, idx in scores[:self.active_k]]

            logger.debug(f"  Module scores: {[(idx, f'{score:.4f}') for score, idx in scores]}")
            return active

        except Exception as e:
            logger.error(f"{RED}Error selecting active modules: {e}{END}")
            # Fallback: return first k modules
            return list(range(self.active_k))

    def _generate_compressed_message(self, state_proj: np.ndarray) -> str:
        """
        Generate compressed coordinator message from projected state.

        In low-rank, message only encodes rank dimensions (much smaller than full state).
        This is the key to reduced communication cost.
        """
        # Simulate message as string representation of compressed state
        message_parts = []
        for i in range(len(state_proj)):
            val = state_proj[i]
            if abs(val) > 0.1:  # Only include significant values
                message_parts.append(f"r{i}:{val:.2f}")

        message = " ".join(message_parts)
        logger.debug(f"  Generated compressed message with {len(message_parts)} components (vs {self.hidden_dim} in full-rank)")
        return message

# Test the low-rank coordinator
method_coordinator = LowRankRecurrentCoordinator(hidden_dim=256, rank=32, num_modules=4)
state, message = method_coordinator.step(sample_outputs)

print(f"🧠 Method coordinator state shape: {state.shape}")
print(f"💬 Coordinator message preview: {message[:100]}...")

# Calculate parameter efficiency
baseline_params = 256 * 256
method_params = 2 * 256 * 32 + 4 * 256 * 32  # U, V matrices + module weights
reduction = (baseline_params - method_params) / baseline_params * 100

print(f"📊 Baseline parameters: {baseline_params:,}")
print(f"📊 Method parameters: {method_params:,}")
print(f"📉 Parameter reduction: {reduction:.1f}%")

## Method: Low-Rank Recurrent Coordinator

The proposed method uses low-rank factorization (W = U @ V^T) with RIM-inspired sparse recurrence to reduce parameter count and communication cost.

In [ ]:
class FullRankCoordinator:
    """Baseline: Full-rank recurrent coordinator for multi-agent coordination."""

    def __init__(self, hidden_dim: int = 256):
        """
        Initialize full-rank coordinator.

        Args:
            hidden_dim: Dimension of hidden state
        """
        self.hidden_dim = hidden_dim
        self.W = np.random.randn(hidden_dim, hidden_dim) * 0.01
        self.state = np.zeros(hidden_dim)

        logger.info(f"{GREEN}FullRankCoordinator initialized: hidden_dim={hidden_dim}{END}")
        logger.info(f"  Weight matrix shape: {self.W.shape}")
        logger.info(f"  Total parameters: {self.W.size}")

    def reset(self):
        """Reset coordinator state."""
        self.state = np.zeros(self.hidden_dim)
        logger.debug("Coordinator state reset")

    def step(self, agent_outputs: List[str]) -> Tuple[np.ndarray, str]:
        """
        Recurrent step: process agent outputs and update state.

        Args:
            agent_outputs: List of agent response strings

        Returns:
            Tuple of (updated state, coordinator message)
        """
        try:
            logger.debug(f"FullRankCoordinator.step with {len(agent_outputs)} agent outputs")

            # Encode agent outputs into feature vector
            features = self._encode_outputs(agent_outputs)
            logger.debug(f"  Encoded features shape: {features.shape}")

            # Full-rank recurrent update: s_{t+1} = W @ s_t + features
            self.state = self.W @ self.state + features
            logger.debug(f"  State norm after update: {np.linalg.norm(self.state):.4f}")

            # Generate coordinator message (simulates communication)
            # In full-rank, message includes all state dimensions
            coordinator_message = self._generate_message(self.state)
            logger.debug(f"  Coordinator message length: {len(coordinator_message)}")

            return self.state.copy(), coordinator_message

        except Exception as e:
            logger.error(f"{RED}Error in FullRankCoordinator.step: {e}{END}")
            raise

    def _encode_outputs(self, outputs: List[str]) -> np.ndarray:
        """Encode agent outputs into feature vector."""
        try:
            # Simple encoding: token counts + basic text statistics
            features = []
            for output in outputs:
                features.extend([
                    len(output.split()),  # Word count
                    len(output),  # Character count
                    output.count('.'),  # Sentence markers
                    output.count('?'),  # Question markers
                ])

            # Pad or truncate to hidden_dim
            features_array = np.array(features[:self.hidden_dim])
            if len(features_array) < self.hidden_dim:
                padded = np.zeros(self.hidden_dim)
                padded[:len(features_array)] = features_array
                features_array = padded

            # Normalize
            features_array = features_array / (np.linalg.norm(features_array) + 1e-8)

            logger.debug(f"  Encoded {len(outputs)} outputs into {len(features_array)} features")
            return features_array

        except Exception as e:
            logger.error(f"{RED}Error encoding outputs: {e}{END}")
            return np.zeros(self.hidden_dim)

    def _generate_message(self, state: np.ndarray) -> str:
        """
        Generate coordinator message from state.

        In full-rank, message encodes entire state (high communication cost).
        """
        # Simulate message as string representation of state
        # Message length proportional to number of dimensions
        message_parts = []
        for i in range(0, len(state), 10):
            # Sample every 10th dimension to create message
            val = state[i]
            if abs(val) > 0.1:  # Only include significant values
                message_parts.append(f"dim{i}:{val:.2f}")

        message = " ".join(message_parts)
        logger.debug(f"  Generated message with {len(message_parts)} components")
        return message

# Test the full-rank coordinator
baseline_coordinator = FullRankCoordinator(hidden_dim=256)
sample_outputs = ["This is response A", "This is response B"]
state, message = baseline_coordinator.step(sample_outputs)

print(f"🧠 Baseline coordinator state shape: {state.shape}")
print(f"💬 Coordinator message preview: {message[:100]}...")
print(f"📊 Parameters: {baseline_coordinator.W.size:,}")

## Baseline: Full-Rank Recurrent Coordinator

The baseline coordinator uses a full-rank recurrent matrix (256×256 parameters) to process agent outputs.

In [ ]:
class TokenTracker:
    """Track token usage for multi-agent coordination."""

    def __init__(self, model: str = "gpt-4"):
        """Initialize token tracker with tiktoken encoder."""
        try:
            self.encoding = tiktoken.encoding_for_model(model)
            logger.info(f"{GREEN}TokenTracker initialized with model: {model}{END}")
        except Exception as e:
            logger.warning(f"{YELLOW}Could not load model-specific encoding, using cl100k_base: {e}{END}")
            self.encoding = tiktoken.get_encoding("cl100k_base")

        self.total_tokens = 0
        self.episode_tokens = []
        self.call_count = 0

    def count_tokens(self, text: str) -> int:
        """Count tokens in text."""
        try:
            tokens = len(self.encoding.encode(text))
            logger.debug(f"Counted {tokens} tokens in text: {truncate_str(text, 50)}")
            return tokens
        except Exception as e:
            logger.error(f"{RED}Error counting tokens: {e}{END}")
            # Fallback: approximate as words * 1.3
            approx = int(len(text.split()) * 1.3)
            logger.warning(f"{YELLOW}Using approximate token count: {approx}{END}")
            return approx

    def log_coordinator_step(self, agent_outputs: List[str], coordinator_message: str = ""):
        """Log tokens for a coordinator step."""
        try:
            step_tokens = 0

            # Count tokens in agent outputs (what coordinator processes)
            for idx, output in enumerate(agent_outputs):
                output_tokens = self.count_tokens(output)
                step_tokens += output_tokens
                logger.debug(f"  Agent {idx} output: {output_tokens} tokens")

            # Count tokens in coordinator's internal message (communication cost)
            if coordinator_message:
                coord_tokens = self.count_tokens(coordinator_message)
                step_tokens += coord_tokens
                logger.debug(f"  Coordinator message: {coord_tokens} tokens")

            self.total_tokens += step_tokens
            self.episode_tokens.append(step_tokens)
            self.call_count += 1

            logger.info(f"{CYAN}Step {self.call_count}: {step_tokens} tokens (total: {self.total_tokens}){END}")
            return step_tokens

        except Exception as e:
            logger.error(f"{RED}Error logging coordinator step: {e}{END}")
            return 0

    def get_stats(self) -> Dict[str, float]:
        """Get aggregated token statistics."""
        stats = {
            "total_tokens": self.total_tokens,
            "num_episodes": len(self.episode_tokens),
            "mean_tokens_per_episode": np.mean(self.episode_tokens) if self.episode_tokens else 0,
            "std_tokens_per_episode": np.std(self.episode_tokens) if self.episode_tokens else 0,
            "call_count": self.call_count
        }
        logger.debug(f"Token stats: {stats}")
        return stats

# Test the token tracker
tracker = TokenTracker()
test_text = "This is a sample text for token counting."
tokens = tracker.count_tokens(test_text)
print(f"🔢 Token count for '{test_text}': {tokens} tokens")

## Token Tracking System

The TokenTracker class monitors communication costs between agents and the coordinator.

In [ ]:
# Define data structures
@dataclass
class ExampleResult:
    """Single example result."""
    input: str
    output: str
    context: Dict[str, Any]
    dataset: str
    split: str
    predict_baseline: str
    predict_method: str
    method: str

@dataclass
class ExperimentResult:
    """Schema matching experiment output format."""
    examples: List[Dict[str, Any]]

# Sample dataset (inlined from JSON files to make notebook self-contained)
SAMPLE_DATASET = {
    "examples": [
        {
            "input": "What is the difference between OpenCL and CUDA?",
            "output": "Winner: model_b",
            "context": {
                "model_a": "chatglm-6b",
                "model_b": "koala-13b",
                "winner": "model_b",
                "judge": "arena_user_973",
                "language": "English",
                "turn": 1,
                "response_a": "OpenCL and CUDA are two different programming models that are used for parallel computing.OpenCL is a general-purpose并行编程接口 that allows developers to write parallel code that can run on any platform that supportsCL, which includes most modern operating systems and computer systems, including Windows, Linux, and macOS. It provides a lower-level, more flexible API that is more suitable for building large-scale distributed computing systems.CUDA is a specific implementation ofOpenCL that is designed for performance and scalability in devices with multiple GPU(s). It was developed by Nvidia and is widely used for scientific computing, machine learning, and图形图像处理. The CUDA API provides a high-level, more efficient way to write parallel code that can run on top of a GPU, while still maintaining the ability to write code that can be easily adapted for other parallel computing platforms.",
                "response_b": "OpenCL and CUDA are both programming languages for parallel computing on GPUs, but they differ in several key ways:\\n\\n1.   Architecture: OpenCL is a general-purpose parallel computing language, while CUDA is designed specifically for GPU computing. OpenCL can run on various types of processors, including CPUs, GPUs, and FPGAs, whereas CUDA is only compatible with NVIDIA GPUs.\\n2.   Language: OpenCL is a C++ language, while CUDA is a C++ language with additional features for GPU computing. CUDA provides additional libraries and tools for handling graphics and other tasks unique to GPUs.\\n3.   Driver support: OpenCL has a wider range of device support, with support for not just NVIDIA GPUs but also other manufacturers like AMD, Intel, and ARM. CUDA is only available on NVIDIA GPUs, and requires the CUDA Toolkit to be installed on the host machine.\\n4.   Memory management: OpenCL provides a unified memory model, where all the data is stored in global memory, and the kernel can access any part of it. CUDA, on the other hand, uses a hierarchical memory architecture, where data is divided into host, device, and shared memory.\\n5.   Performance: OpenCL and CUDA have different strengths and weaknesses in terms of performance. OpenCL is generally faster for numerical computations, while CUDA is faster for graphics and other tasks that require fast memory access. CUDA also has better support for 64-bit floating-point operations, which are essential for some scientific applications.\\n\\nIn summary, OpenCL and CUDA are both programming languages for parallel computing on GPUs, but they differ in terms of architecture, language, driver support, memory management, and performance. OpenCL is a general-purpose language that can run on various types of processors, while CUDA is specific to NVIDIA GPUs and provides additional libraries and tools for handling graphics and other tasks unique to GPUs."
            },
            "dataset": "lmsys/chatbot_arena_conversations",
            "split": "train"
        },
        {
            "input": "Why did my parent not invite me to their wedding?",
            "output": "Winner: tie",
            "context": {
                "model_a": "oasst-pythia-12b",
                "model_b": "alpaca-13b",
                "winner": "tie",
                "judge": "arena_user_973",
                "language": "English",
                "turn": 1,
                "response_a": "It is possible that your parent did not invite you to their wedding for a variety of reasons. It could be that they did not want to burden you with the planning or expense of the event, or they may not have had the time or resources to include you. Additionally, it could be that they did not feel comfortable having you attend due to your age or relationship with them. It is important to remember that your parent may have had their own reasons for not inviting you, and it is not your place to judge their choices. If you feel neglected or upset about not being invited, it is best to talk to them about it and express your feelings in a non-accusatory way.",
                "response_b": "It is likely that they wanted to keep the guest list small and intimate. Maybe they felt that inviting you would make the event too crowded, or that they wanted to keep the event reserved for close friends and family. It could be that they wanted to avoid any potential drama or conflict that could arise from having too many people at the event. Whatever the reason, it's important to remember that they made this decision out of love and respect for you, and that their decision doesn't define your relationship."
            },
            "dataset": "lmsys/chatbot_arena_conversations",
            "split": "train"
        },
        {
            "input": "Fuji vs. Nikon, which is better?",
            "output": "Winner: model_b",
            "context": {
                "model_a": "koala-13b",
                "model_b": "oasst-pythia-12b",
                "winner": "model_b",
                "judge": "arena_user_973",
                "language": "English",
                "turn": 1,
                "response_a": "Both Fuji and Nikon are popular camera brands that offer a wide range of cameras for different types of photography. It really depends on your specific needs and preferences as to which brand is \\\"better\\\" for you.\\n\\nFujifilm is known for its high image quality, especially in the areas of color accuracy and dynamic range. Many photographers also appreciate the company's focus on ergonomics and the overall user experience of its cameras. Additionally, Fujifilm has a loyal following among enthusiasts and professional photographers, thanks to its commitment to film photography and its high-end mirrorless cameras.\\n\\nNikon, on the other hand, is known for its advanced autofocus system and image stabilization technology, as well as its high-end cameras and lenses. Nikon also has a wide selection of cameras and lenses to choose from, including both DSLR and mirrorless options.\\n\\nUltimately, it's difficult to say which brand is \\\"better\\\" without knowing more about your specific needs and preferences. It might be helpful to research both brands further and try out a few different models before making a decision.",
                "response_b": "This is a subjective question and the answer depends on the individual preferences and needs. Both Fujifilm and Nikon are well-known for their high-quality cameras, and the choice between the two ultimately depends on the user's personal preferences and needs.\\n\\nSome factors to consider when deciding between a Fuji and a Nikon camera include:\\n\\n1. Image quality: Fuji cameras are known for their high-quality images, with a distinctive film-like quality. Nikon cameras are also notable for their image quality, but they may not offer the same level of sharpness and color accuracy as Fuji cameras.\\n\\n2. Focusing: Fuji cameras are known for their fast and precise autofocus, while Nikon cameras tend to have a slower and more manual focus system.\\n\\n3. Image size: Fuji cameras are known for their wide range of photo sizes, from small compacts to large-format cameras. Nikon cameras are also capable of producing high-quality images, but they may not be as versatile as Fuji cameras.\\n\\n4. User interface: Fuji cameras have a user-friendly interface, with a simplified design that is easy to navigate. Nikon cameras may have a more complex interface, but they also offer a wealth of features and customization options.\\n\\nUltimately, the best camera for you will depend on your specific needs and preferences. If you're looking for a high-quality camera that offers a unique film-like experience, a Fuji camera may be the right choice for you. If you're looking for a camera that is easy to use and offers a wide range of photo sizes, a Nikon camera may be the best choice."
            },
            "dataset": "lmsys/chatbot_arena_conversations",
            "split": "train"
        }
    ]
}

print(f"📁 Dataset loaded: {len(SAMPLE_DATASET['examples'])} examples")
print(f"🎯 Example structure: {list(SAMPLE_DATASET['examples'][0].keys())}")
print(f"🔍 First question: {SAMPLE_DATASET['examples'][0]['input']}")

## Data Schema and Sample Dataset

We use data from the lmsys/chatbot_arena_conversations dataset with multi-agent interactions. Each example contains agent responses that need to be coordinated.

In [ ]:
# Import required libraries
import json
import logging
import sys
from pathlib import Path
from typing import Dict, List, Tuple, Any
from dataclasses import dataclass, asdict

import numpy as np
import tiktoken
from scipy.linalg import svd
from sklearn.metrics import accuracy_score, f1_score
from scipy.stats import ttest_rel

# Configure logging for interactive use
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s | %(levelname)-7s | %(funcName)-20s | %(message)s'
)
logger = logging.getLogger(__name__)

# Color codes for enhanced output
BLUE, GREEN, YELLOW, CYAN, RED, END = "\033[94m", "\033[92m", "\033[93m", "\033[96m", "\033[91m", "\033[0m"

def truncate_str(text: str, max_len: int = 100) -> str:
    """Truncate long strings for logging."""
    if len(text) <= max_len:
        return text
    return text[:max_len] + f"... ({len(text)} chars total)"

print("✅ Libraries imported successfully!")
print("📊 Ready to analyze low-rank recurrent coordinators")

# Empirical Evaluation of Low-Rank Recurrent Coordinator on Multi-LLM Communication Efficiency

This notebook demonstrates a low-rank recurrent coordinator for multi-LLM agent communication efficiency. The experiment compares a baseline full-rank coordinator against a low-rank coordinator with RIM-inspired sparse recurrence.

## Key Components
1. **Baseline**: Full-rank recurrent coordinator (256×256 parameters)
2. **Method**: Low-rank recurrent coordinator (rank=32, 4 RIM modules, 25% parameters)
3. **Evaluation**: Token efficiency vs task performance on multi-agent interactions

## Hypothesis
Low-rank coordinator reduces token usage by >15% while maintaining task performance.